In [435]:
# 모듈을 불러온다. 
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import konlpy
import nltk
from konlpy.tag import Kkma;kkma = Kkma()
from konlpy.corpus import kobill

In [95]:
pol_articles = ticon(pol_articles)
cul_articles = ticon(cul_articles)
sci_articles = ticon(sci_articles)

In [1]:
#--------------NewsCrawling------------------#
# 특정 기사의 본문과 제목을 가져오는 함수. 
def NewsCraw(url):
    html = urlopen(url) # url에서 html을 가져온다. 
    soup = BeautifulSoup(html,"html.parser", from_encoding = "utf-8") # bs4에서 파싱할 수 있는 구조로 바꾼다.
    content = soup.find(id = 'articleBodyContents') # id로 본문 내용을 가져온다. 
    hangul = re.compile("[^ ㄱ-ㅎ|가-힣.""]+") # 한글,.,띄어쓰기가 아닌 문자(영어, 특수문자, 숫자 등등)를 나타내는 정규표현식을 만든다.
    content = str(content) # html 객체?를 문자열로 바꾼다.
    content = hangul.sub("",content) # 정규표현식에 해당하는 패턴을 지운다. 
    content = re.sub("본문 내용  플레이어   플레이어    오류를 우회하기 위한 함수 추가","",content) # 불필요한 한글 패턴 제거
    content = re.sub('본문 내용  플레이어   동영상 뉴스       영상 플레이어   플레이어    오류를 우회하기 위한 함수 추가','',content)
    content = re.sub("본문 내용  플레이어   동영상 뉴스 .      영상 플레이어   플레이어    오류를 우회하기 위한 함수 추가",'',content)
    title = soup.find("h3",{"id":"articleTitle"}).find(text=True) # 제목을 가져온다. 
    return [title ,content] # 제목과 내용을 반환한다. 

# 기사목록에서 개별기사를 가져오는 함수.
def articles(url):
    articles = []
    for i in range(1,6):
        page_url = url+str(i) # i를 문자열로 변경.
        page_html = urlopen(page_url) 
        page_soup = BeautifulSoup(page_html,"html.parser",from_encoding = "utf-8")
        articles.extend(page_soup.findAll("dt"))
        url_list = []
        for link in articles:
            url_list.append(link.find("a")["href"])
            url_list = list(set(url_list))
    return url_list

# 기사에서 제목과 내용을 가져오는 함수.
def ticon(url_list):
    df = DataFrame(columns=("기사제목","본문내용"))
    for i, url in enumerate(url_list):
        df.loc[i] = NewsCraw(url)
    return df

# 뉴스 주제별로 기사목록을 가져온다. 
pol_url = "https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20181009&page=" 
cul_url = "https://news.naver.com/main/list.nhn?mode=LS2D&sid2=245&sid1=103&mid=shm&date=20181009&page="
sci_url = "https://news.naver.com/main/list.nhn?mode=LS2D&sid2=228&sid1=105&mid=shm&date=20181009&page="

# 토픽에 따라 기사들을 정리. 
pol_articles = articles(pol_url)
cul_articles = articles(cul_url)
sci_articles = articles(sci_url)

pol_articles = ticon(pol_articles)
cul_articles = ticon(cul_articles)
sci_articles = ticon(sci_articles)

In [245]:
#--------------LDA------------------#
# Tokenize
pol_doc = []
for i in range(0,len(pol_articles)):
    pol_doc.append(kkma.morphs(pol_articles.iloc[i,1]))

cul_doc = []
for i in range(0,len(cul_articles)):
    cul_doc.append(kkma.morphs(cul_articles.iloc[i,1]))

sci_doc = []
for i in range(0,len(sci_articles)):
    sci_doc.append(kkma.morphs(sci_articles.iloc[i,1]))

In [248]:
len(pol_doc) #100

100

In [249]:
len(cul_doc) #100

100

In [250]:
len(sci_doc) #33

33

In [228]:
# LDA
import random
from collections import Counter

In [254]:
documents = []

In [255]:
documents.extend(pol_doc)

In [257]:
documents.extend(cul_doc)

In [258]:
documents.extend(sci_doc)

In [259]:
len(documents) # 1~100까지는 정치 101~200까지는 문화 200~33은 과학

233

In [261]:
#조건부 확률 분포 정의를 위한 준비

#topic의 개수
K = 3

#1. 각 토픽이 각 문서에 할당되는 횟수
#counter로 구성된 list
#각각의 counter는 각 문서를 의미함
document_topic_counts = [Counter() for _ in documents]

#2. 각 단어가 각 토픽에 할당되는 횟수
# 각각의 counter는 각 토픽을 의미함
topic_word_counts = [Counter() for _ in range(K)] 

#3. 각 토픽에 할당되는 총 단어 수
# 각각의 숫자는 각 토픽을 의미함
topic_counts = [0 for _ in range(K)] 

#4. 각 문서에 포함되는 총 단어의 수
# 각각의 숫자는 각 문서를 의미함
document_lengths = [len(d) for d in documents]

#5. 단어 종류의 수
distinct_words = set(word for document in documents for word in document) 
W = len(distinct_words)

#6. 총 문서의 수 
D = len(documents)

#### 새로운 topic 계산하기

In [262]:
def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

In [264]:
def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

In [265]:
def topic_weight(d, word, k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

In [266]:
def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

#랜덤으로 생성된 weight로부터 인덱스를 생성함
def sample_from(weights):
     total = sum(weights)
     rnd = total * random.random()       # uniform between 0 and total
     for i, w in enumerate(weights):
         rnd -= w                        # return the smallest i such that
         if rnd <= 0: return i           # sum(weights[:(i+1)]) >= rnd

In [267]:
random.seed(0)

#topic의 개수
K = 3

# 각 단어를 임의의 토픽에 배정
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

# 랜덤 초기화한 상태에서 AB를 구하는 데 필요한 숫자 계산하기
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [269]:
# 조건부 확률 분포를 이용하여 (토픽-단어), (문서-토픽)에 대한 깁스 샘플링 실행하기
for iter in range(1000): 
    print(iter)
    for d in range(D): 
        for i, (word, topic) in enumerate(zip(documents[d], 
                                              document_topics[d])): 
 
 
           # remove this word / topic from the counts
           # so that it doesn't influence the weights 
            document_topic_counts[d][topic] -= 1 
            topic_word_counts[topic][word] -= 1 
            topic_counts[topic] -= 1 
            document_lengths[d] -= 1 
 
           # choose a new topic based on the weights 
            new_topic = choose_new_topic(d, word) 
            document_topics[d][i] = new_topic 

 
           # and now add it back to the counts 
            document_topic_counts[d][new_topic] += 1 
            topic_word_counts[new_topic][word] += 1 
            topic_counts[new_topic] += 1 
            document_lengths[d] += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [270]:
#각 토픽에 가장 영향력이 높은 (weight)값이 큰 단어 탐색
for k, word_counts in enumerate(topic_word_counts): 
         for word, count in word_counts.most_common(): 
             if count > 0: print (k, word, count) 

0 하 1227
0 . 922
0 이 803
0 ㄴ 756
0 는 742
0 다 645
0 을 624
0 에 494
0 의 444
0 를 402
0 어 401
0 었 401
0 은 364
0 ㄹ 269
0 으로 265
0 있 253
0 되 242
0 가 221
0 고 213
0 에서 203
0 로 190
0 과 189
0 도 167
0 등 144
0 연구 136
0 지 124
0 수 122
0 것 116
0 ... 112
0 아 106
0 ㄴ다 98
0 기술 94
0 개 92
0 와 91
0 들 89
0 교수 87
0 일 85
0 게 84
0 개발 84
0 .. 83
0 기 83
0 과학 77
0 미생물 68
0 뉴스 67
0 팀 66
0 부터 65
0 김 64
0 및 63
0 분석 63
0 국내 62
0 환자 62
0 서울 61
0 않 59
0 겠 58
0 위하 57
0 병 57
0 산업 57
0 한국 56
0 까지 53
0 견 52
0 호텔 51
0 면 50
0 억 50
0 정보 48
0 밝히 47
0 기업 47
0 만 47
0 대 46
0 원 45
0 기자 44
0 사람 44
0 확인 44
0 바이오 44
0 노 43
0 내용 42
0 본문 41
0 통하 40
0 내 40
0 미국 39
0 토마토 39
0 월 38
0 받 37
0 비 37
0 열 37
0 분야 37
0 바로 37
0 경제 36
0 부 36
0 더 36
0 에게 36
0 ㄴ다는 36
0 영웅 36
0 배포 35
0 오 35
0 만들 35
0 연합 35
0 물질 35
0 위원 34
0 결과 34
0 사용 34
0 제공 34
0 낮 34
0 다는 33
0 이용 33
0 통신 32
0 식물 32
0 많 31
0 분리 31
0 발생 31
0 암 31
0 의료 31
0 가기 30
0 금지 30
0 니 30
0 전재 30
0 았 30
0 마름 30
0 풋 30
0 올해 29
0 데 29
0 재 29
0 해외 29
0 동아 29
0 저항성 29
0 뿌리 28
0 일본 28
0 장비 28
0 이번 27


0 심의 3
0 중점 3
0 단편 3
0 쌀쌀 3
0 해상 3
0 바다 3
0 농도 3
0 션 3
0 분의 3
0 준수 3
0 조각 3
0 확대 3
0 선택 3
0 오브 3
0 특징 3
0 딱 3
0 썰 3
0 획득 3
0 사이트 3
0 경복궁 3
0 북서쪽 3
0 유입 3
0 평년 3
0 먼바다 3
0 날씨 3
0 약하 3
0 일종 3
0 살아가 3
0 노골적 3
0 데뷔 3
0 보컬 3
0 보드 3
0 언더 3
0 뮤직 3
0 친 3
0 발 3
0 상대적 3
0 력 3
0 결혼 3
0 오픈 3
0 가득 3
0 시골 3
0 주인공 3
0 수년 3
0 암세포 3
0 수많 3
0 로렌스 3
0 버클리 3
0 잔디 3
0 복용 3
0 무용지물 3
0 이롭 3
0 떠올리 3
0 제어 3
0 지름 3
0 관찰 3
0 스페 3
0 유사성 3
0 자로 3
0 바이러스 3
0 유전 3
0 병의 3
0 가지과 3
0 어린이 3
0 현미경 3
0 사인 3
0 엑스 3
0 요소 3
0 혁명 3
0 세미나 3
0 전부 3
0 감마선 3
0 트랜지스터 3
0 다기능 3
0 평안 3
0 침입 3
0 박막 3
0 회로 3
0 미리 3
0 향서 3
0 비영리 3
0 최호 3
0 불가피 3
0 변신 3
0 솔라 3
0 국산화하 3
0 역량 3
0 시상 3
0 레 3
0 시들 3
0 대항 3
0 머신 3
0 저장 3
0 러닝 3
0 지능 3
0 펙사벡 3
0 고르 3
0 액 3
0 빗물 3
0 공대 3
0 유도 3
0 여섯 3
0 학술 3
0 유관 3
0 스테이지 3
0 광학계 3
0 고조도 3
0 벗어나 3
0 셀 3
0 반사 3
0 훨씬 3
0 모션 3
0 아버지 3
0 라디칼 3
0 농업 3
0 군집 3
0 강제 3
0 수금 3
0 장점 3
0 고추 3
0 세대 3
0 고소 3
0 항만 3
0 여대 3
0 진심 3
0 메탄 3
0 결함 3
0 가정 3
0 토록 3
0 육성 3
0 충원 3
0 슈퍼스타 3
0 불가능 3
0 클라우드 3
0 세포주 3
0 거주 3
0 식물체 3
0 거르 

0 고뇌 1
0 사색 1
0 석해 1
0 적나라 1
0 까치발 1
0 쓸어내리 1
0 실질 1
0 사의 1
0 실화 1
0 비커 1
0 인간성 1
0 리벤구드 1
0 허물어지 1
0 참여로 1
0 태환 1
0 제위 1
0 오렌지색 1
0 태양계 1
0 신비 1
0 이대 1
0 탄수화물 1
0 궤도 1
0 외계 1
0 생체 1
0 상거래 1
0 법률안 1
0 비축 1
0 인체 1
0 세균성 1
0 분석법 1
0 다드 1
0 대기권 1
0 탑재 1
0 닙은 1
0 사체 1
0 복지부 1
0 텍스 1
0 석학 1
0 하겐 1
0 공화국 1
0 마땅 1
0 대원동 1
0 몰딩 1
0 가공 1
0 점유율 1
0 분말 1
0 구현 1
0 제너럴 1
0 일렉트릭 1
0 직스 1
0 점도 1
0 변성 1
0 스의 1
0 딩스 1
0 계열사 1
0 상용 1
0 존엄 1
0 넘어서 1
0 총액 1
0 다고만 1
0 효력 1
0 행위자 1
0 옛 1
0 합작 1
0 컨소시엄 1
0 폐쇄 1
0 전경 1
0 키예프 1
0 지점 1
0 질환 1
0 플릭시맵 1
0 킬러 1
0 고혈압 1
0 기한 1
0 특정인 1
0 심인 1
0 므로 1
0 테라젠이텍스 1
0 팩 1
0 파멥신 1
0 식각 1
0 천연원료 1
0 마이트 1
0 차단기 1
0 오염물 1
0 무영 1
0 요르단 1
0 송편 1
0 신흥 1
0 혼합 1
0 자매지 1
0 고속도로 1
0 음전기 1
0 전기적 1
0 부서 1
0 면적 1
0 생산성 1
0 글라스 1
0 비추 1
0 여종 1
0 액세사리 1
0 불통 1
0 게임 1
0 대외 1
0 달인 1
0 청소기 1
0 야적 1
0 표상 1
0 고름 1
0 강지 1
0 광복회 1
0 청 1
0 나본 1
0 원저자 1
0 장의 1
0 토론 1
0 싫 1
0 기류 1
0 라야 1
0 고급 1
0 보틀넥 1
0 예기 1
0 한전 1
0 주범 1
0 체계화 1
0 현수막 1
0 매달 1
0 수십만 1
0 꼬박꼬박 1
0 정위 1
0 생사 1
0 품 1
0 씻 1
0 남강댐 1


1 불편 6
1 여기 6
1 요금 6
1 같이 6
1 걷 6
1 나서 6
1 기름 6
1 눈길 6
1 떠오르 6
1 한겨레 6
1 음모 6
1 자백 6
1 폰 6
1 바꾸 6
1 약속 6
1 이탈리아 6
1 롤 6
1 선거 6
1 울 6
1 내년 6
1 상식 6
1 대단히 6
1 마지막 6
1 낳 6
1 시세표 6
1 방한 6
1 재단 6
1 능력 6
1 완전 6
1 버크 6
1 향하 5
1 거론 5
1 승 5
1 현지 5
1 렉 5
1 주재 5
1 환경 5
1 홍보 5
1 튜브 5
1 예고 5
1 관광 5
1 얼굴 5
1 반 5
1 더라도 5
1 ㄴ다는 5
1 국무총리 5
1 체제 5
1 환대 5
1 기로 5
1 전문가 5
1 보여주 5
1 저유소 5
1 콘텐츠 5
1 실시 5
1 충분 5
1 표명 5
1 돈 5
1 개미 5
1 못하 5
1 아요 5
1 대 5
1 반응 5
1 께 5
1 거든요 5
1 화면 5
1 남측 5
1 구 5
1 고는 5
1 차관 5
1 마련 5
1 실제 5
1 못 5
1 쉽 5
1 인기 5
1 제가 5
1 세우 5
1 제시 5
1 우려 5
1 쌀 5
1 있다 5
1 제재 5
1 투데이 5
1 작용 5
1 근 5
1 얼마나 5
1 입 5
1 고르 5
1 주요 5
1 주문 5
1 잠시 5
1 방식 5
1 제국 5
1 임종 5
1 유료 5
1 공명 5
1 카 5
1 반장 5
1 아무튼 5
1 키워드 5
1 꺼리 5
1 고문 5
1 거기 5
1 무슨 5
1 일어나 5
1 군요 5
1 허용 5
1 트로 5
1 원장 5
1 전례 5
1 토 5
1 사상 5
1 수행원 5
1 찾아오 5
1 옆 5
1 부각 5
1 접하 5
1 디자인 5
1 할아버지 5
1 소리 5
1 들리 5
1 서류 5
1 스스로 5
1 역대 5
1 회복 5
1 맺 5
1 변화 5
1 한우 5
1 실현 5
1 장기수 5
1 전향 5
1 웃 5
1 끝나 5
1 문의 5
1 노드 5
1 항구적 5
1 계시 5
1 진술서 5
1 자세히 5
1 동시 5
1 앞 4
1 터키 4
1 개회사

1 뜨 1
1 때리 1
1 확립 1
1 회장 1
1 상징 1
1 케니 1
1 아티스트 1
1 고시 1
1 리쿠 1
1 사인 1
1 품 1
1 서해 1
1 남자 1
1 신곡 1
1 자연 1
1 사랑가 1
1 전립선 1
1 펜 1
1 파스타 1
1 전립 1
1 활성 1
1 항산 1
1 분비 1
1 몰려들 1
1 몰리 1
1 늘리 1
1 동석 1
1 페스티벌 1
1 외치 1
1 낯설 1
1 비밀 1
1 안타깝 1
1 남양주시 1
1 계신 1
1 손님 1
1 이슬 1
1 자음 1
1 종성 1
1 사투리 1
1 말랑말랑 1
1 조상 1
1 전래 1
1 동화 1
1 데스 1
1 벤츠 1
1 패션 1
1 개방 1
1 교구 1
1 커리큘럼 1
1 시스템 1
1 절차 1
1 갱신 1
1 출발 1
1 참여도 1
1 어느 1
1 쿠 1
1 콩 1
1 산 1
1 부스럭대 1
1 잔잔히 1
1 어쩜 1
1 옷 1
1 주무시 1
1 원통 1
1 볼록 1
1 입꼬리 1
1 속이 1
1 구시렁거리 1
1 컴 1
1 짐짓 1
1 사무적 1
1 떠올리 1
1 참 1
1 돌보 1
1 토닥이 1
1 초등학교 1
1 창호 1
1 머리 1
1 여자 1
1 팽팽 1
1 잊어버리 1
1 한산 1
1 온통 1
1 혼몽 1
1 뒷이야기 1
1 차갑 1
1 반기 1
1 빡 1
1 뉘우스 1
1 잼 1
1 툰 1
1 위령제 1
1 발길 1
1 아마추어 1
1 바둑계 1
1 승부수 1
1 백은 1
1 매치 1
1 랩 1
1 앤드 1
1 미라 1
1 남부 1
1 소재 1
1 하이라이트 1
1 사물 1
1 좌 1
1 군민 1
1 과감 1
1 합 1
1 사도 1
1 맞이하 1
1 공산당 1
1 지루 1
1 작전 1
1 여태껏 1
1 아버지 1
1 허망 1
1 출판 1
1 만류 1
1 이분 1
1 갇히 1
1 희성 1
1 천대 1
1 살아오 1
1 묘 1
1 타이완 1
1 량 1
1 돌아가시 1
1 모토 1
1 보라 1
1 척하 1
1 꼬집 1
1 취미 1
1 조문 1
1 조연출 1
1 남친 1
1 진지 1
1 라이크

2 체제 11
2 딸 11
2 군 11
2 국무총리 11
2 선동 11
2 현황 11
2 여론 11
2 드 11
2 진 11
2 취소 11
2 왕의 11
2 한국어 11
2 베 11
2 교통부 11
2 교섭 11
2 집행 11
2 행위 11
2 추가 11
2 부분 11
2 아서 11
2 기금 11
2 방명록 11
2 세 11
2 교체 11
2 내년 11
2 강 11
2 규 11
2 인식 11
2 동의 11
2 출입 11
2 스 11
2 반대 11
2 위반 11
2 셈 11
2 갈등 11
2 아직 11
2 작가 11
2 소리 11
2 은행 11
2 창작 11
2 아리랑 11
2 개혁 10
2 조차 10
2 ㅂ니다 10
2 빠지 10
2 백성 10
2 김정숙 10
2 몇 10
2 실적 10
2 개정 10
2 보법 10
2 제가 10
2 역사 10
2 듣 10
2 막 10
2 권고 10
2 근절 10
2 성 10
2 점검 10
2 민주주의 10
2 광 10
2 다가 10
2 양 10
2 여사 10
2 마치 10
2 발전 10
2 국제선 10
2 모습 10
2 선 10
2 모으 10
2 넘 10
2 갈 10
2 해 10
2 울산 10
2 고민 10
2 역시 10
2 참가 10
2 부인 10
2 힘들 10
2 까 10
2 속 10
2 법안 10
2 명예 10
2 확보 10
2 여전히 10
2 실형 10
2 율 10
2 순간 10
2 쇼 10
2 영진 10
2 대로 10
2 ㄹ라 10
2 류 10
2 없다 10
2 혜 10
2 저녁 9
2 니 9
2 으며 9
2 여당 9
2 기본 9
2 최종 9
2 광장 9
2 일정 9
2 장관 9
2 분 9
2 인물 9
2 아야 9
2 포함 9
2 언론 9
2 시절 9
2 영주 9
2 해병대 9
2 생활 9
2 종합 9
2 중단 9
2 방침 9
2 광화문 9
2 겨레말 9
2 팝 9
2 위대 9
2 총 9
2 방지 9
2 조 9
2 지시 9
2 놓 9
2 백 9
2 칼 9
2 거나 9
2 돈 9
2 고용 9
2 갖 9
2 민족 9
2 

2 으므로 2
2 자도 2
2 심규 2
2 인의 2
2 환산 2
2 원의 2
2 병식 2
2 애족장 2
2 친 2
2 회견 2
2 영릉으 2
2 분향 2
2 왕릉 2
2 상지 2
2 창조적 2
2 고등학교 2
2 도가 2
2 영릉에 2
2 악수 2
2 그런데 2
2 선물 2
2 철저히 2
2 설립 2
2 치유 2
2 결산 2
2 룸살롱 2
2 결제 2
2 부의 2
2 출국 2
2 불구 2
2 제주 2
2 혼잡 2
2 승객 2
2 격리 2
2 요인 2
2 벨트 2
2 항공 2
2 반영 2
2 비밀 2
2 편하 2
2 부족 2
2 파스 2
2 진짜 2
2 우선 2
2 표시 2
2 상봉 2
2 사무소 2
2 일어나 2
2 엮 2
2 맞춤 2
2 늦추 2
2 격 2
2 보기 2
2 공이 2
2 도약 2
2 신륵사 2
2 지정 2
2 링 2
2 철칙 2
2 부문 2
2 강도 2
2 매체 2
2 의도 2
2 견제 2
2 끊이 2
2 추 2
2 박근 2
2 는다는 2
2 민주적 2
2 속앓이 2
2 전무 2
2 실시간 2
2 이메일 2
2 장인 2
2 찰스 2
2 강좌 2
2 잠들 2
2 커 2
2 자재비 2
2 산후 2
2 조리 2
2 공금 2
2 직후 2
2 동지 2
2 영입 2
2 면면 2
2 인적 2
2 윤상 2
2 당파 2
2 점프 2
2 서면 2
2 해방 2
2 확 2
2 차장 2
2 별다르 2
2 문건 2
2 압수 2
2 중순 2
2 서비스 2
2 상품 2
2 이른바 2
2 자재 2
2 예방 2
2 추산 2
2 진행형 2
2 균형 2
2 는다고 2
2 외교 2
2 선출 2
2 자당 2
2 합의점 2
2 지난주 2
2 동료 2
2 부산시 2
2 정하 2
2 정청 2
2 유체 2
2 불이 2
2 고는 2
2 생활관 2
2 소초 2
2 사관 2
2 인간 2
2 사상 2
2 또한 2
2 막아주 2
2 네요 2
2 몇몇 2
2 붐 2
2 토픽 2
2 권한 2
2 본사 2
2 따로 2
2 유흥 2
2 베를린 2
2 다소 2
2 펼치 2
2 기독 2
2 침해 2
2 선전 

2 소홀히 1
2 속보 1
2 쟁 1
2 점화 1
2 시정 1
2 월간 1
2 비축미 1
2 동질성 1
2 어로 1
2 잡아내 1
2 최대한 1
2 알차 1
2 사적 1
2 여지 1
2 효능 1
2 떨어뜨리 1
2 뛰어넘 1
2 경선 1
2 자발적 1
2 잔류 1
2 화물기 1
2 부대변인 1
2 서재 1
2 충당 1
2 본관 1
2 대중음악 1
2 막판 1
2 타결 1
2 맞느냐 1
2 보조비 1
2 해결책 1
2 포용 1
2 비록 1
2 연일 1
2 더군요 1
2 의총 1
2 주관적 1
2 는군요 1
2 걷어붙이 1
2 법명 1
2 소수자 1
2 여권 1
2 정정 1
2 상징물 1
2 무력화 1
2 포털 1
2 공통점 1
2 불구하 1
2 접점 1
2 첫발 1
2 획정 1
2 지지부진 1
2 재수 1
2 동남 1
2 장사 1
2 범행 1
2 노장 1
2 심층 1
2 절단할 1
2 우가 1
2 전쟁터 1
2 연주가 1
2 금석문 1
2 장년층 1
2 현진건 1
2 이슬기 1
2 단오 1
2 남궁 1
2 직결 1
2 한국적 1
2 내보내 1
2 남용 1
2 예쁘 1
2 화두 1
2 휴무 1
2 예사 1
2 증대 1
2 로비 1
2 랭보 1
2 가구 1
2 간직 1
2 사이좋 1
2 붙잡 1
2 만족감 1
2 내적 1
2 미적 1
2 추첨 1
2 전산망 1
2 집단적 1
2 신상 1
2 극장주 1
2 낭독 1
2 테 1
2 구석 1
2 핑계 1
2 앨 1
2 터울 1
2 자마자 1
2 보조 1
2 어르신 1
2 헤치 1
2 캐스팅 1
2 자전거 1
2 서명 1
2 심리적 1
2 미스 1
2 만나고 1
2 골든아워 1
2 뉴웨이브 1
2 드물 1
2 가인 1
2 마스터 1
2 강태 1
2 앞뒤 1
2 히트곡 1
2 손짓 1
2 화음 1
2 팅스 1
2 최우수 1
2 톱스타 1
2 그래미 1
2 카자흐 1
2 아프리카 1
2 파크 1
2 축소 1
2 윤리적 1
2 수첩 1
2 촛불 1
2 출판물 1
2 차병원 1
2 관악구 1
2 유학생 1
2 지을 1
2 상진 1

In [484]:
topic_names = ["정치", "문화", "과학"]

df = DataFrame(columns=("토픽","횟수","문서번호"))
num = 0 
for document, topic_counts in zip(documents, document_topic_counts): 
        num += 1
        for topic, count in topic_counts.most_common():
            temp = pd.DataFrame({"토픽":topic_names[topic],"횟수":count,"문서번호":num}, index = [0])
            df = df.append(temp)

In [488]:
df = df.

In [489]:
df

,토픽,횟수,문서번호
0,문화,317,1
0,과학,127,1
0,정치,6,1
0,문화,157,2
0,과학,0,2
0,정치,0,2
0,과학,274,3
0,문화,195,3
0,정치,0,3
0,문화,393,4
